## Comparative analysis of Nanjing Climate in 1969, 1996, 2023

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TkAgg')  # Set before importing pyplot for GUI-based display
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Read all NetCDF files (1996, 1969, and 2023)
nc_file_1996 = '2m_temperature_nanjing.nc'
nc_file_1969 = '2m_temperature_nanjing_1969.nc'
nc_file_2023 = '2m_temperature_nanjing_2023.nc'

ds_1996 = xr.open_dataset(nc_file_1996)
ds_1969 = xr.open_dataset(nc_file_1969)
ds_2023 = xr.open_dataset(nc_file_2023)

# Function to extract Nanjing temperature data
def extract_nanjing_temp(ds):
    # Nanjing coordinates: 32.06°N, 118.79°E
    lat_idx = np.argmin(np.abs(ds.latitude.values - 32.06))
    lon_idx = np.argmin(np.abs(ds.longitude.values - 118.79))
    nanjing_temp = ds.t2m.isel(latitude=lat_idx, longitude=lon_idx)

    # Convert to DataFrame and process
    df = nanjing_temp.to_dataframe().reset_index()
    df = df.set_index('valid_time')
    df = df[['t2m']]
    df.columns = ['temperature']
    df['temperature'] = df['temperature'] - 273.15  # Kelvin to Celsius
    return df

# Extract data for all three years (1996, 1969, 2023)
df_1996 = extract_nanjing_temp(ds_1996)
df_1969 = extract_nanjing_temp(ds_1969)
df_2023 = extract_nanjing_temp(ds_2023)

# Add year column for visualization
df_1996['year'] = 1996
df_1969['year'] = 1969
df_2023['year'] = 2023

# Create a common time index (day of year)
df_1996['day_of_year'] = df_1996.index.dayofyear
df_1969['day_of_year'] = df_1969.index.dayofyear
df_2023['day_of_year'] = df_2023.index.dayofyear

# Plot all three years together
plt.figure(figsize=(15, 6))

# Plot temperature fluctuation for 1996 (blue)
plt.plot(df_1996['day_of_year'], df_1996['temperature'], label='1996', color='blue', alpha=0.7)

# Plot temperature fluctuation for 1969 (red)
plt.plot(df_1969['day_of_year'], df_1969['temperature'], label='1969', color='red', alpha=0.7)

# Plot temperature fluctuation for 2023 (green)
plt.plot(df_2023['day_of_year'], df_2023['temperature'], label='2023', color='green', alpha=0.7)

# Adding titles and labels
plt.title('Nanjing Temperature Comparison: 1969, 1996, and 2023')
plt.xlabel('Day of Year')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.grid(True)

# Add month labels to x-axis
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
month_starts = [1, 32, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]

# Set x-ticks to show the start of each month
plt.xticks(month_starts, months, rotation=45)

([<matplotlib.axis.XTick at 0x1f9a53816d0>,
 [Text(1, 0, 'Jan'),
  Text(32, 0, 'Feb'),
  Text(60, 0, 'Mar'),
  Text(91, 0, 'Apr'),
  Text(121, 0, 'May'),
  Text(152, 0, 'Jun'),
  Text(182, 0, 'Jul'),
  Text(213, 0, 'Aug'),
  Text(244, 0, 'Sep'),
  Text(274, 0, 'Oct'),
  Text(305, 0, 'Nov'),
  Text(335, 0, 'Dec')])

In [2]:
# Adjust the x-axis tick intervals for better spacing
from matplotlib.ticker import MaxNLocator

# Set x-axis locator to have fewer ticks and wider spacing (e.g., every 30 days or based on data range)
plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True, prune='both'))

# Ensure the plot layout is tight and labels are displayed properly
plt.tight_layout()

# Show the plot using Matplotlib
plt.show()

In [ ]:
# Now proceed with LSTM modeling for one of the years (1996 in this case)
# We'll use df_1996 for the LSTM model as in the original code

# Preprocess the data: normalize using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df_1996[['temperature']])

# Function to create sequences for time series prediction
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:(i + seq_length), 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

# Create sequences with length 3 for prediction
seq_length = 3  # Use 3 time steps to predict the next one
X, y = create_sequences(scaled_data, seq_length)

# Split the data into training and testing sets
# Use the last 10 days (240 hours) as the test set
train_size = len(X) - 240
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Convert numpy arrays to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

# Create DataLoader for batch processing during training
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define the LSTM model (same as before)
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=1, output_size=1):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out

# Initialize the model, loss function, and optimizer
model = LSTM(input_size=1, hidden_size=50, num_layers=1, output_size=1)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X.unsqueeze(2))
        loss = criterion(outputs, batch_y.unsqueeze(1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make predictions
model.eval()
with torch.no_grad():
    train_predict = model(X_train.unsqueeze(2)).squeeze().numpy()
    test_predict = model(X_test.unsqueeze(2)).squeeze().numpy()

# Inverse transform the predictions and actual values
train_predict = scaler.inverse_transform(train_predict.reshape(-1, 1))
y_train = scaler.inverse_transform(y_train.numpy().reshape(-1, 1))
test_predict = scaler.inverse_transform(test_predict.reshape(-1, 1))
y_test = scaler.inverse_transform(y_test.numpy().reshape(-1, 1))

# Calculate Root Mean Squared Error (RMSE)
train_rmse = np.sqrt(np.mean((train_predict - y_train) ** 2))
test_rmse = np.sqrt(np.mean((test_predict - y_test) ** 2))
print(f'Train RMSE: {train_rmse:.2f}')
print(f'Test RMSE: {test_rmse:.2f}')

# Visualize the LSTM results for 1996
plt.figure(figsize=(15, 6))
plt.plot(df_1996.index[-240:], y_test, label='Actual')
plt.plot(df_1996.index[-240:], test_predict, label='Predicted')
plt.title('LSTM Model: Actual vs Predicted Temperature for Last 10 Days (1996)')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.grid(True)
plt.show()